<a href="https://colab.research.google.com/github/JoseColombini/PCS5022-deeplearning/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to
Basics to import files from github

### TODO
- Generalizar para todas as tasks

In [14]:
import requests
import numpy as np
from io import BytesIO

url = 'https://github.com/JoseColombini/PCS5022-deeplearning/raw/refs/heads/main/dataset1.npz'
response = requests.get(url)

# Check status
if response.status_code == 200:
    data = np.load(BytesIO(response.content))

    # Now you can access arrays inside your .npz file, e.g.:
    print(data.files)  # lists arrays inside the archiveelse:
    print(data['X_train'])
else:
    print("Failed to fetch file:", response.status_code)

['X_train', 'y_train', 'X_val', 'y_val', 'X_test']
[[1.   0.   1.   ... 0.   0.   0.  ]
 [1.   0.   1.   ... 0.   0.   0.  ]
 [1.   0.   1.   ... 0.   0.   0.  ]
 ...
 [0.   0.   0.   ... 2.   2.   7.98]
 [1.   0.   1.   ... 0.   0.   0.  ]
 [0.   0.   0.   ... 1.   1.   4.29]]


# Gerador de CSV para submissão

- use **Y_predict** to save the outputs of your model. It will automatically fit it here to generate the csv

- Be sure you are sending the csv of the correct task

In [47]:
def run_submission(model, x_test):
    model.eval()
    y_test = []

    # Convert x_test to a torch tensor if it's a numpy array or list
    if isinstance(x_test, (list, np.ndarray)):
        x_test_t = torch.tensor(x_test, dtype=torch.float32)
    else:
        x_test_t = x_test.float()

    for x in x_test_t:
      # Ensure it's 2D: (1, input_size)
      if x.ndim == 1:
          x = x.unsqueeze(0)

      with torch.no_grad():
          output = model(x)
          prob = torch.sigmoid(output).item()
          prediction = 1.0 if prob >= 0.5 else 0.0

          # Append the prediction
          y_test.append(prediction)

    return y_test

y_test = run_submission(model, X_test)
print(y_test)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0,

In [48]:
import numpy as np
import pandas as pd

# Carrega a base de dados a partir de seu caminho
# data = np.load(dataset)
# X_test = data['X_test']

# Faz palpites aleatórios
num_samples = X_test.shape[0]
# random_predictions = np.random.randint(0, 2, size=num_samples)

# Gera um DataFrame no formato esperado da submissão
submission_df = pd.DataFrame({
    'ID': np.arange(1, num_samples + 1),
    'Prediction': y_test
})

# Salva o arquivo CSV no diretório atual
submission_df.to_csv('submission.csv', index=False)

# Task 1

### TODO
- Dividir em varios blocos menores e talvez reutilizar para outras tasks
- Fazer um modelo de verdade
- refatorar

In [44]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

X_train = data['X_train']
Y_train = data['y_train']
print(X_train.shape[1])
print(Y_train)


print(data)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(119, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 4),
            nn.ReLU(),
            nn.Linear(4, 2),
            nn.ReLU(),
            nn.Linear(2, 1),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


# Convert numpy arrays to torch tensors
X_train_t = torch.tensor(X_train, dtype=torch.float32)
Y_train_t = torch.tensor(Y_train, dtype=torch.float32)  # float for BCEWithLogitsLoss

# Dataset and DataLoader
train_dataset = TensorDataset(X_train_t, Y_train_t)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Instantiate model
model = NeuralNetwork()

# Use BCEWithLogitsLoss for binary classification
loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for batch_X, batch_Y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X).squeeze()  # (batch_size,) shape

        loss = loss_fn(outputs, batch_Y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * batch_X.size(0)

    epoch_loss = running_loss / len(train_dataset)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")




Using cuda device
119
[1 1 1 ... 0 1 1]
NpzFile 'object' with keys: X_train, y_train, X_val, y_val, X_test
Epoch 1/10, Loss: 0.5539
Epoch 2/10, Loss: 0.5383
Epoch 3/10, Loss: 0.5236
Epoch 4/10, Loss: 0.5253
Epoch 5/10, Loss: 0.5159
Epoch 6/10, Loss: 0.5131
Epoch 7/10, Loss: 0.5109
Epoch 8/10, Loss: 0.5091
Epoch 9/10, Loss: 0.5073
Epoch 10/10, Loss: 0.5062


In [45]:
x_val = data['X_val']
y_val = data['y_val']
X_val_t = torch.tensor(x_val, dtype=torch.float32)
y_val_t = torch.tensor(y_val, dtype=torch.float32)

def evaluate(model, X, y, batch_size=32):
    model.eval()
    val_dataset = torch.utils.data.TensorDataset(X, y)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)

    loss_fn = nn.BCEWithLogitsLoss()
    total_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            outputs = model(batch_X).squeeze()
            loss = loss_fn(outputs, batch_y)
            total_loss += loss.item() * batch_X.size(0)

            # Get predicted labels: sigmoid + threshold at 0.5
            probs = torch.sigmoid(outputs)
            preds = (probs >= 0.5).float()
            correct += (preds == batch_y).sum().item()
            total += batch_X.size(0)

    avg_loss = total_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

val_loss, val_accuracy = evaluate(model, X_val_t, y_val_t)
print(f"Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.2%}")

Validation Loss: 0.7165, Accuracy: 64.52%


In [46]:
X_test = data['X_test']